Because of convention, a markdown forward literate kernel must be an option.  The literate kernel may be an aggressive opinion.  In this post, we'll discuss levels of commitment to a literate kernel using different parts of the `IPython` system.

In [1]:
import sys

In [2]:
Markdown forward literate computing permits Markdown as source code, and the block/inline code objects are executed.  The different ways of using Markdown as code ordered from least to most opinionated:

1. a <code>%%markdown</code> or <code>%%md</code> cell magic that an author voluntarily invokes.
2. using markdown as a fallback.  It is very easy for Markdown to appear as a `SyntaxError` to Python, and in this case the Markdown transformer is invoked.
3. a permenant `IPython.core.inputtransformer` that is executed each time a cell is evaluated.
4. a custom `ipykernel` that says "I want literate computing all the damn time".

Markdown forward literate computing permits Markdown as source code, and the block/inline code objects are executed.  The different ways of using Markdown as code ordered from least to most opinionated:

1. a <code>%%markdown</code> or <code>%%md</code> cell magic that an author voluntarily invokes.
2. using markdown as a fallback.  It is very easy for Markdown to appear as a `SyntaxError` to Python, and in this case the Markdown transformer is invoked.
3. a permenant `IPython.core.inputtransformer` that is executed each time a cell is evaluated.
4. a custom `ipykernel` that says "I want literate computing all the damn time".


------> SyntaxError
------> IPython.core.inputtransformer
------> ipykernel


NameError: name 'IPython' is not defined

In [3]:
ip = get_ipython()

In [4]:
import abc

In [5]:
from deathbeds.__SyntaxError_Fallbacks import fallback

In [41]:
import traceback

In [45]:

from contextlib import contextmanager

In [46]:
@contextmanager
def swap_exception(Exception):
    ip.custom_exceptions = tuple(x for x in ip.custom_exceptions if not issubclass(x, Exception)); yield
    ip.custom_exceptions += (Exception,)

In [50]:
import fnmatch

In [78]:
class BaseExceptions(abc.ABCMeta):
    def __call__(self, ip, type, Exception, tb, **kwargs):
        excepts = getattr(self, type.__name__, None)
        if excepts:
            with swap_exception(type):
                return getattr(self, type.__name__)(ip, type, Exception, tb, **kwargs)
class Exceptions(metaclass=BaseExceptions): 
    SyntaxError = fallback
    
    def NameError(ip, type, Exception, tb, **kwargs):
        msg = Exception.args[0]
        pat = "name '*' is not defined"
        if fnmatch.fnmatch(msg, pat):
            left, right = pat.split('*')
            result = ip.run_cell(f"import {msg[len(left):-len(right)]}", store_history=False)
            if not any((result.error_in_exec, result.error_before_exec)):
                result = ip.run_cell(In[-1])
        return traceback.format_exception_only(type, Exception)
    
    
    def AttributeError(ip, type, Exception, tb, **kwargs):
        msg = Exception.args[0]
        pat = "module '*' has no attribute '*'"
        if fnmatch.fnmatch(msg, pat):
            left, middle, right = pat.split('*')
            mstart = msg.index(middle)
            mend = mstart + len(middle)
            result = ip.run_cell(f"import {msg[len(left):mstart]}.{msg[mend:-len(right)]}", store_history=False)
            if not any((result.error_in_exec, result.error_before_exec)):
                result = ip.run_cell(In[-1])
        return traceback.format_exception_only(type, Exception)


In [79]:
ip.set_custom_exc(tuple(getattr(__builtin__, x) for x in dir(Exceptions) if x not in dir(abc.ABC)), Exceptions)

In [81]:
sklearn.preprocessing

<module 'sklearn.preprocessing' from 'C:\\Anaconda3\\lib\\site-packages\\sklearn\\preprocessing\\__init__.py'>

In [72]:
IPython

<module 'IPython' from 'C:\\Anaconda3\\lib\\site-packages\\IPython\\__init__.py'>

In [37]:
There is a syntax

    print(11)

There is a syntax

    print(11)


------> print(11)


11


In [19]:
Exceptions(get_ipython(), BaseException, BaseException(), BaseException().__traceback__)

In [ ]:
ip.set_custom_exc